**Connect google drive**

In [0]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Thu May  7 17:23:18 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8    25W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': Operation not supported
'A Project Report.gdoc'
'Colab Notebooks'
 commandslinux330.gdoc
'Copy of Startup Inspiration Black by Slidesgo.gslides'
'GENERAL PROFICIENCY-STUDENTS EVALUATION_2019 (1) (1).gdoc'
'GENERAL PROFICIENCY-STUDENTS EVALUATION_2019 (1).docx'
'GENERAL PROFICIENCY-STUDENTS EVALUATION_2019 (1).gdoc'
'Getting started.pdf'
'Live Long-3.gslides'
'Live Long-3.pptx'
'major project ppt - Copy.gslides'
'major project ppt - Copy.pptx'
'New Microsoft Word Document (1).docx'
'New Microsoft Word Document (1).gdoc'
'New Microsoft Word Document (2).gdoc'
'New Microsoft Word Document.docx'
'New Microsoft Word Document.gdoc'
 PRE-ASSESS_Candidate_Details-2020-20191113.xlsx
'report major project1.docx'
'report major project1.gdoc'
'shuffle_1208- 27 Jun 2019__06-27-2019_18h_44m_42s.csv'
'shuffle_1208- 2

**1) Clone the Darknet**



In [0]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 13357 (delta 8), reused 13 (delta 4), pack-reused 13331
Receiving objects: 100% (13357/13357), 12.04 MiB | 11.87 MiB/s, done.
Resolving deltas: 100% (9114/9114), done.


**2) Compile Darknet using Nvidia GPU**


In [0]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:910:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1391:14: warning: unused variable ‘buff’ [-Wunused-variable]
         char buff[100];
              ^~~~
./src/image_opencv.cpp:1367:9: warning: unused variabl

**3) Configure Darknet network for training YOLO V3**

In [0]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [0]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [0]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [0]:
!echo "number_plate" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [0]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-05-07 17:26:02--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   449KB/s    in 10m 52s 

2020-05-07 17:36:56 (243 KB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [0]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/7.jpg          
  inflating: data/obj/7.txt          
  inflating: data/obj/8.jpg          
  inflating: data/obj/8.txt          
  inflating: data/obj/9.jpg          
  inflating: data/obj/9.txt          
  inflating: data/obj/10.jpg         
  inflating: data/obj/10.txt         
  inflating: data/obj/11.jpg         
  inflating: data/obj/11.txt         
  inflating: data/obj/12.jpg         
  inflating: data/obj/12.txt         
  inflating: data/obj/13.jpg         
  inflating: data/obj/13.txt         
  inflating: data/obj/14.jpg         
  inflating: data/obj/14.txt         
  inflating: data/obj/15.jpg         
  inflating: data/obj/15.txt         
  inflating: data/obj/16.jpg         
  inflating: data/obj/16.txt         
  inflating: data/obj/17.jpg         
  inflating: data/obj/17.txt         
  inflating: data/obj/18.jpg         
  inflating: data/obj/18.txt         
  inflating: data/obj/19.jpg         
  inflating: 

In [0]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

0 0.10510510510510501 0.39 0.576576576576576 0.544
['0', '0.10510510510510501', '0.39', '0.576576576576576', '0.544']
0 data/obj/63.txt
0 0.10510510510510501 0.39 0.576576576576576 0.544
0 0.052208835341365 0.48953974895397406 0.5903614457831321 0.778242677824267
['0', '0.052208835341365', '0.48953974895397406', '0.5903614457831321', '0.778242677824267']
1 data/obj/231.txt
0 0.052208835341365 0.48953974895397406 0.5903614457831321 0.778242677824267
0 0.373046875 0.485330073349633 0.509765625 0.540342298288508
['0', '0.373046875', '0.485330073349633', '0.509765625', '0.540342298288508']
2 data/obj/56.txt
0 0.373046875 0.485330073349633 0.509765625 0.540342298288508
0 0.28600000000000003 0.689265536723163 0.678 0.9067796610169491
['0', '0.28600000000000003', '0.689265536723163', '0.678', '0.9067796610169491']
3 data/obj/214.txt
0 0.28600000000000003 0.689265536723163 0.678 0.9067796610169491
0 0.3359375 0.427586206896551 0.6828124999999999 0.549425287356321
['0', '0.3359375', '0.42758620

In [0]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/42.jpg', 'data/obj/173.jpg', 'data/obj/132.jpg', 'data/obj/192.jpg', 'data/obj/139.jpg', 'data/obj/9.jpg', 'data/obj/128.jpg', 'data/obj/19.jpg', 'data/obj/179.jpg', 'data/obj/88.jpg', 'data/obj/100.jpg', 'data/obj/34.jpg', 'data/obj/94.jpg', 'data/obj/66.jpg', 'data/obj/233.jpg', 'data/obj/193.jpg', 'data/obj/97.jpg', 'data/obj/134.jpg', 'data/obj/59.jpg', 'data/obj/27.jpg', 'data/obj/207.jpg', 'data/obj/195.jpg', 'data/obj/75.jpg', 'data/obj/215.jpg', 'data/obj/210.jpg', 'data/obj/40.jpg', 'data/obj/63.jpg', 'data/obj/37.jpg', 'data/obj/163.jpg', 'data/obj/183.jpg', 'data/obj/191.jpg', 'data/obj/185.jpg', 'data/obj/222.jpg', 'data/obj/156.jpg', 'data/obj/111.jpg', 'data/obj/113.jpg', 'data/obj/180.jpg', 'data/obj/162.jpg', 'data/obj/114.jpg', 'data/obj/121.jpg', 'data/obj/199.jpg', 'data/obj/80.jpg', 'data/obj/47.jpg', 'data/obj/25.jpg', 'data/obj/65.jpg', 'data/obj/50.jpg', 'data/obj/229.jpg', 'data/obj/38.jpg', 'data/obj/230.jpg', 'data/obj/203.jpg', 'data/obj/167.jpg', 

In [0]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [0]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.391180, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 1367.057983, iou_loss = 0.000000, total_loss = 1367.057983 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.466525, GIOU: 0.452667), Class: 0.456692, Obj: 0.570139, No Obj: 0.544022, .5R: 0.500000, .75R: 0.000000, count: 4, class_loss = 160.114166, iou_loss = 0.906113, total_loss = 161.020279 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.479898, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 499.435303, iou_loss = 0.000000, total_loss = 499.435303 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.390163, .5R: 0.000000, .75R: 0.00